### Imports

In [1]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, load_model
from keras.layers.recurrent import GRU
from keras.layers import Dense, LSTM, Bidirectional

from encrypt import *
from tools import *
import string

Using TensorFlow backend.


### Configurations

In [2]:
letters = string.printable.split('!')[0]
encrypt = shifting_cipher
samples_count = 200000
text_length = 10

set_characters(letters)
model_path = f"{encrypt.__name__}_{len(letters)}x{text_length}_best_model.h5"

### Generating and Preparing Data

In [3]:
text = generate_text(text_length, samples_count)
cipher = list(map(encrypt, text))

train_text = to_vec(text)
train_cipher = to_vec(cipher)

### Building and Training the Model

In [4]:
model = Sequential()
model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=train_cipher.shape[1:]))
model.add(Dense(1))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 10, 256)           99840     
_________________________________________________________________
dense_1 (Dense)              (None, 10, 1)             257       
Total params: 100,097
Trainable params: 100,097
Non-trainable params: 0
_________________________________________________________________


In [5]:
callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=3, verbose=1, mode='min'),
    ModelCheckpoint(model_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
]
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.fit(train_cipher, train_text, validation_split=.1, batch_size=10, epochs=5, callbacks=callbacks)

Train on 180000 samples, validate on 20000 samples
Epoch 1/5
180000/180000 [==============================] - 148s 823us/step - loss: 9.6683 - acc: 0.7396 - val_loss: 0.0307 - val_acc: 0.9903

Epoch 00001: val_acc improved from -inf to 0.99029, saving model to shifting_cipher_62x10_best_model.h5
Epoch 2/5
180000/180000 [==============================] - 185s 1ms/step - loss: 0.1573 - acc: 0.9550 - val_loss: 0.0345 - val_acc: 0.9635

Epoch 00002: val_acc did not improve from 0.99029
Epoch 3/5
180000/180000 [==============================] - 173s 961us/step - loss: 0.0334 - acc: 0.9798 - val_loss: 0.0237 - val_acc: 0.9959

Epoch 00003: val_acc improved from 0.99029 to 0.99588, saving model to shifting_cipher_62x10_best_model.h5
Epoch 4/5
180000/180000 [==============================] - 167s 929us/step - loss: 0.0237 - acc: 0.9870 - val_loss: 0.0033 - val_acc: 1.0000

Epoch 00004: val_acc improved from 0.99588 to 1.00000, saving model to shifting_cipher_62x10_best_model.h5
Epoch 5/5
18000

### Predicting and Evaluating the Model

In [6]:
model = load_model(model_path)

In [7]:
test_cyhper = encrypt('HelloWorld')
prediction = to_txt(model.predict(to_vec(test_cyhper)))

print('Model predicted that original text is:', prediction)
print('Test cipher vs encrypted prediction:', test_cyhper, 'vs', encrypt(prediction))

Model predicted that original text is: HelloWorld
Test cipher vs encrypted prediction: elloWorldH vs elloWorldH
